## How to use

In this noteboook we test the rewards given out by the protocol to different types of validators. Our `fast` config reduces the size of most constants to avoid allocating more memory than necessary (we'll only test with a few validators). We also reduce the number of slots per epoch to speed things up. All these changes are without loss of generality.

You may test various spec versions by renaming either of the `specs_*.py` files in the `beaconrunner` folder to `specs.py`. Current version tested is v1.1.0-alpha.5-PR2453.

In [1]:
import importlib
import types
from eth2spec.config.config_util import prepare_config
from eth2spec.utils.ssz.ssz_impl import hash_tree_root

import os, sys
# Load current beacon runner specs (v1.1.0-alpha.3)
sys.path.insert(1, os.path.realpath(os.path.pardir))

import beaconrunner as br

prepare_config(".", "fast.yaml")

br.reload_package(br)

Below are metrics extracted from the state of the simulation. We don't really care about most of them except for `get_current_validator_state`, but they are left here for future use/reference.

In [2]:
def extract_state(s):
    validators = s["network"].validators
    validator = validators[1]
    head = br.specs.get_head(validator.store)
    current_state = validator.store.block_states[head].copy()
    return current_state

def current_slot(params, step, sL, s, _input):
    return ("current_slot", s["network"].validators[0].data.slot)

def total_balance_asap(params, step, sL, s, _input):
    validators = s["network"].validators
    current_state = extract_state(s)
    current_epoch = br.specs.get_current_epoch(current_state)
    asap_indices = [i for i, v in enumerate(validators) if v.validator_behaviour == "asap"]
    asap_balances = [b for i, b in enumerate(current_state.balances) if i in asap_indices]
    return ("total_balance_asap", sum(asap_balances))

def get_base_reward(params, step, sL, s, _input):
    current_state = extract_state(s)
    base_reward = br.specs.get_base_reward(current_state, 0)
    return ("base_reward", base_reward)

def get_block_proposer(params, step, sL, s, _input):
    current_state = extract_state(s)
    block_proposer = [v.validator_index for v in s["network"].validators if v.data.current_proposer_duties[s["current_slot"] % br.specs.SLOTS_PER_EPOCH]][0]
    return ("block_proposer", block_proposer)

def get_block_proposer_balance(params, step, sL, s, _input):
    current_state = extract_state(s)
    block_proposer_balance = current_state.balances[s["block_proposer"]]
    return ("block_proposer_balance", block_proposer_balance)

def get_sync_committee(params, step, sL, s, _input):
    current_state = extract_state(s)
    current_epoch = br.specs.get_current_epoch(current_state)
    sync_committee = current_state.current_sync_committee
    sync_committee = br.specs.get_sync_committee_indices(current_state, current_epoch)
    return ("sync_committee", sync_committee)

def get_head(params, step, sL, s, _input):
    validators = s["network"].validators
    validator = validators[0]
    head = br.specs.get_head(validator.store).hex()[0:6]
    return ("head", head)

def get_current_validator_state(params, step, sL, s, _input):
    current_state = extract_state(s)
    current_validator_state = []
    for v in s["network"].validators:
        current_validator_state += [{
            "slot": v.data.slot,
            "validator_index": v.validator_index,
            "balance": current_state.balances[v.validator_index],
            "block_proposer": 1 if s["block_proposer"] == v.validator_index else 0,
            "attester": 1 if v.data.current_attest_slot == v.data.slot else 0,
            "sync_committee": len(v.data.current_sync_committee),
        }]
    return ("current_validator_state", current_validator_state)

observers = {
    "current_slot": current_slot,
    "total_balance_asap": total_balance_asap,
    "base_reward": get_base_reward,
    "block_proposer": get_block_proposer,
    "block_proposer_balance": get_block_proposer_balance,
#     "sync_committee": get_sync_committee,
    "head": get_head,
    "current_validator_state": get_current_validator_state,
}

Run the simulation.

In [3]:
from random import sample
from beaconrunner.validators.ASAPValidator import ASAPValidator

num_validators = 16

# Initiate validators
validators = []
for i in range(num_validators):
    new_validator = ASAPValidator(i)
    validators.append(new_validator)

# Create a genesis state
(genesis_state, genesis_block) = br.simulator.get_genesis_state_block(validators)

# Validators load the state
[v.load_state(genesis_state.copy(), genesis_block.copy()) for v in validators]

# We skip the genesis block
br.simulator.skip_genesis_block(validators)

# Create the network
set_a = br.network.NetworkSet(validators=list(range(num_validators)))
network_sets = list([set_a])
network = br.network.Network(validators = validators, sets = network_sets)

# Set simulation parameters
parameters = br.simulator.SimulationParameters({
    "num_epochs": 4,
    "run_index": 1,
    "frequency": 1,
    "network_update_rate": 1.0,
})

df = br.simulator.simulate(network, parameters, observers) 

will simulate 4 epochs ( 16 slots ) at frequency 1 moves/second
total 192 simulation steps

                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (192, 2, 1, 8)


Initializing configurations:   0%|          | 0/1 [00:00<?, ?it/s]

Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
11 proposing block for slot 1
11 proposing block for slot 2
6 proposing block for slot 3
5 proposing block for slot 4
9 proposing block for slot 5
0 proposing block for slot 6
0 proposing block for slot 7
12 proposing block for slot 8
7 proposing block for slot 9
timestep 100 of run 1
13 proposing block for slot 10
10 proposing block for slot 11
2 proposing block for slot 12
0 proposing block for slot 13
4 proposing block for slot 14
9 proposing block for slot 15
4 proposing block for slot 16


Flattening results:   0%|          | 0/769 [00:00<?, ?it/s]

Total execution time: 90.05s


In [4]:
df = df.drop(columns=['network', 'simulation', 'subset', 'run'])

We collect the validator state data in a separate dataframe, of schema `(slot, validator_index, balance, block_proposer, attester, sync_committee)`. We ignore the first few epochs (rewards aren't given out, or imperfectly) and make sure the key `(slot, validator_index)` is unique.

In [5]:
import itertools
import pandas as pd
pd.set_option('display.max_rows', 100)
validator_df = pd.DataFrame(
    itertools.chain(*df[((df.substep == 0) | (df.substep == 4)) & (df.timestep % 12 == 1) & (df.current_slot > 11)]["current_validator_state"].iloc[:])
)
validator_df["previous_balance"] = validator_df.groupby(["validator_index"])[["balance"]].shift()
validator_df["balance_diff"] = validator_df["balance"] - validator_df["previous_balance"]
validator_df = validator_df[validator_df.slot > 12]
validator_df

,slot,validator_index,balance,block_proposer,attester,sync_committee,previous_balance,balance_diff
16,13,0,32009011262,1,0,1,3.200754e+10,1475787.0
17,13,1,32000626094,0,1,0,3.200063e+10,0.0
18,13,2,32005657199,0,1,0,3.200566e+10,0.0
19,13,3,32004718064,0,0,1,3.200467e+10,44721.0
20,13,4,32000626094,0,0,0,3.200063e+10,0.0
21,13,5,32005657199,0,1,0,3.200566e+10,0.0
22,13,6,32002057160,0,0,0,3.200206e+10,0.0
23,13,7,32006149130,0,0,1,3.200610e+10,44721.0
24,13,8,32001118025,0,0,1,3.200107e+10,44721.0
25,13,9,32006149130,0,0,1,3.200610e+10,44721.0


## Obtaining rewards per duty

We isolate validators who occupied a unique role over a slot to obtain the rewards for block proposers and sync committee members. When rewards are given out during the epoch transition, the minimum received by any validator is received by a validator who neither proposed a block for the whole epoch and was never part of the sync committee, giving us the attester reward received by one validator.

In [6]:
block_proposer_reward = validator_df[(validator_df.block_proposer == 1) & (validator_df.sync_committee == 0)].iloc[0]["balance_diff"]
print(f"block proposer reward = {block_proposer_reward}")

block proposer reward = 1431066.0


In [7]:
attester_reward = min(validator_df[validator_df.slot % br.specs.SLOTS_PER_EPOCH == 0]["balance_diff"])
print(f"attester reward = {attester_reward}")

attester reward = 2414934.0


In [8]:
sync_committee_reward = validator_df[(validator_df.block_proposer == 0) & (validator_df.sync_committee == 1)].iloc[0]["balance_diff"]
print(f"sync committee reward = {sync_committee_reward}")

sync committee reward = 44721.0


In [9]:
base_reward = df.iloc[0]["base_reward"]
print(f"base reward = {base_reward}")

base reward = 2862144


Make our checks.

In [10]:
total_per_epoch = num_validators * base_reward
total_for_proposers = int(br.specs.SLOTS_PER_EPOCH) * block_proposer_reward
total_for_attesters = num_validators * attester_reward
total_for_sync_committee = int(br.specs.SLOTS_PER_EPOCH) * int(br.specs.SYNC_COMMITTEE_SIZE) * sync_committee_reward
print(f"expected total per epoch = {total_per_epoch} vs. realised total = {total_for_proposers + total_for_attesters + total_for_sync_committee}")
print(f"percent of attester rewards = {total_for_attesters / total_per_epoch}")
print(f"percent of sync committee rewards = {total_for_sync_committee / total_per_epoch}")
print(f"percent of block proposer rewards = {total_for_proposers / total_per_epoch}")

expected total per epoch = 45794304 vs. realised total = 45794280.0
percent of attester rewards = 0.84375
percent of sync committee rewards = 0.03125
percent of block proposer rewards = 0.12499947591735426
